In [114]:
import pandas as pd
from pprint import pprint
import numpy as np
from os import listdir
from os.path import isfile, join
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import log_loss
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')

In [154]:
def select_frame(csv, single=True):
    # Take csv where each row is a frame in the video and return
    # either 1 still frame or multiple still frames
    df = pd.read_csv(csv)
    print(len(df))
    deltas = []
    for i in range(len(df)-1,0,-1):
        # iterate in reverse order as best poses often last
        current_ = df.iloc[i]
        next_ = df.iloc[i-1]
        deltas.append(np.mean(abs(next_ - current_)))
    SD = np.std(deltas)
        
    if single:
        # Single Frame
        for i in range(len(df)-1,0,-1):
            # iterate in reverse order as best poses often last
            current_ = df.iloc[i]
            next_ = df.iloc[i-1]
            d = np.mean(abs(next_ - current_)) 
            if d < 0.05*SD:
                print(i)
                return df.iloc[i]
    else:
        # Multiple Frames
        ds = deltas < SD*0.5
        return df[:-1][ds]

In [156]:
def mean_ten_still_frames(pose_csv):
    pose_df = pd.read_csv(pose_csv)
    pose_diff = pose_df.diff()
    rows_total_diff = pose_diff.sum(axis=1)
    rows_total_diff = [abs(i) for i in rows_total_diff]
    ten_rows_diff = []
    for i in range(len(rows_total_diff)-10):
        ten_rows_diff.append((i, sum(rows_total_diff[i:(i+10)])))
    best_ten = sorted(ten_rows_diff, key=lambda x: x[1], reverse=False)
    still_point = best_ten[0][0]
    stillest_ten = pose_df.iloc[still_point:still_point+10, :]
    mean = np.mean(stillest_ten, axis=0) 
    return mean[1:]

In [162]:
def ten_still_frames(pose_csv):
    pose_df = pd.read_csv(pose_csv)
    pose_diff = pose_df.diff()
    rows_total_diff = pose_diff.sum(axis=1)
    rows_total_diff = [abs(i) for i in rows_total_diff]
    ten_rows_diff = []
    for i in range(len(rows_total_diff)-10):
        ten_rows_diff.append((i, sum(rows_total_diff[i:(i+10)])))
    best_ten = sorted(ten_rows_diff, key=lambda x: x[1], reverse=False)
    still_point = best_ten[0][0]
    stillest_ten = pose_df.iloc[still_point:still_point+10, :]
    return stillest_ten

In [157]:
mean_ten_still_frames("train_data_csv/warrior2_arms_1.csv")

0     683.488200
1     151.372800
2       0.890713
3     635.277800
4     214.290600
5       0.872670
6     573.007900
7     214.274500
8       0.766997
9     479.134400
10    262.849600
11      0.723817
12    394.279700
13    298.683100
14      0.802875
15    697.955700
16    214.469300
17      0.738082
18    781.415600
19    209.693700
20      0.756055
21    867.197800
22    192.097700
23      0.800729
24    635.268200
25    398.221400
26      0.707443
27    588.275200
28    398.214100
29      0.665110
         ...    
45    678.303700
46    136.675000
47      0.857739
48    684.294300
49    140.754800
50      0.194423
51    635.517500
52    136.494800
53      0.899878
54      0.000000
55      0.000000
56      0.000000
57    852.764300
58    635.344000
59      0.633255
60    852.093900
61    633.284600
62      0.557444
63    776.330700
64    631.308000
65      0.755177
66    378.608600
67    648.921200
68      0.803391
69    366.837100
70    637.247600
71      0.773194
72    386.4424

In [168]:
def x_y_points(data):
    """
    from openpose 'pose_keypoints_2d' array, 
    finds x and y corridnates and return two lists
    """
    x_warrior = []
    y_warrior = []
    c_warrior = [] # certainity of pose
    keypoints = data['people'][0]['pose_keypoints_2d']
    for n in range(len(keypoints)):
        if (n%3) == 0:
            x_warrior.append(keypoints[n])
        elif (n%3) == 1:
            y_warrior.append(keypoints[n])
        elif (n%3) == 1:
            c_warrior.append(keypoints[n])
    
    return x_warrior, y_warrior

In [169]:
x, y = x_y_points(mean_ten_still_frames("train_data_csv/warrior2_arms_1.csv"))

KeyError: 'people'

In [167]:
ten_still_frames("train_data_csv/warrior2_arms_5.csv")

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,65,66,67,68,69,70,71,72,73,74
271,274,711.693,133.663,0.953380,668.588,208.175,0.845147,621.605,206.193,0.741700,...,0.826836,464.862,648.881,0.855207,455.017,639.152,0.798531,468.740,617.680,0.649773
272,275,711.695,133.679,0.947900,668.583,208.177,0.845161,621.599,206.196,0.740375,...,0.826889,464.874,648.882,0.853748,455.034,639.156,0.799887,468.733,617.675,0.653401
273,276,711.689,133.682,0.945772,668.588,208.178,0.843372,621.602,206.201,0.736899,...,0.825990,466.740,648.879,0.853373,455.043,639.172,0.796971,468.708,617.682,0.650862
274,277,711.695,133.668,0.947961,668.597,208.176,0.842845,623.480,206.190,0.738021,...,0.829734,464.873,648.887,0.857922,455.030,639.181,0.796403,466.797,617.678,0.653129
275,278,711.698,133.695,0.947044,668.600,208.186,0.842013,623.478,206.201,0.737150,...,0.822281,464.875,648.902,0.857284,455.035,639.202,0.790740,466.801,617.681,0.650041
276,279,711.691,133.678,0.949176,668.599,208.194,0.844299,623.488,206.202,0.736130,...,0.823297,464.867,648.884,0.852806,455.018,639.184,0.791152,468.702,619.523,0.648654
277,280,711.689,133.648,0.949716,668.584,208.191,0.846324,623.492,206.205,0.733079,...,0.827541,464.869,648.866,0.850059,455.019,639.166,0.791757,468.705,619.538,0.654434
278,281,711.695,133.691,0.947052,668.600,210.012,0.843916,623.527,206.225,0.729773,...,0.827534,464.866,648.870,0.845938,455.008,639.173,0.785470,468.721,619.534,0.650949
279,282,711.693,133.692,0.954116,668.578,210.019,0.851641,623.498,208.094,0.734908,...,0.828645,464.880,648.877,0.847849,455.031,639.175,0.789966,466.807,617.686,0.649424
280,283,711.703,133.688,0.954214,668.581,210.013,0.853156,623.500,206.233,0.736434,...,0.818464,464.878,648.871,0.849406,455.035,639.166,0.792815,466.804,617.689,0.654895


In [158]:
labels_df = pd.read_csv("labels-single_label_per_row.csv")

In [159]:
labels_df.head()

,name,label
0,warrior2_knee1_1,front_knee_acute
1,warrior2_knee1_2,front_knee_acute
2,warrior2_knee1_3,front_knee_acute
3,warrior2_knee1_3,torso_forward
4,warrior2_knee1_4,front_knee_acute


In [160]:
singles_df = pd.DataFrame()

for name in labels_df['name']:
    if name != "warrior2_correct_1":
        dfs = select_frame("train_data_csv/" + name + ".csv")
        singles_df[name] = dfs

727
723
518
517
535
534
535
534
895
888
895
888
581
579
581
579
329
327
329
327
956
954
956
954
634
632
634
632
840
836
840
836
629
627
387
325
387
325
709
694
515
514
515
514
828
825
599
588
599
588
599
588
364
321
364
321
958
956
958
956
563
532
563
532
881
880
881
880
413
325
413
325
413
325
413
325
369
363
369
363
827
826
827
826
827
826
448
447
448
447
448
447
448
447
925
924
925
924
925
924
925
924
483
481
483
481
483
481
483
481
467
466
467
466
467
466
467
466
682
677
456
453
1265
1257
1327
1326


In [161]:
singles_df 

,warrior2_knee1_1,warrior2_knee1_2,warrior2_knee1_3,warrior2_knee1_4,warrior2_knee1_5,warrior2_knee1_6,warrior2_knee2_1,warrior2_knee2_2,warrior2_knee2_3,warrior2_knee2_4,...,warrior2_torso_5,warrior2_hips_1,warrior2_hips_2,warrior2_hips_3,warrior2_hips_4,warrior2_hips_5,warrior2_correct_2,warrior2_correct_3,warrior2_correct_4,warrior2_correct_5
Unnamed: 0,748.000000,524.000000,534.000000,888.000000,581.000000,335.000000,963.000000,633.000000,836.000000,645.000000,...,363.000000,834.000000,449.000000,924.000000,483.000000,469.000000,680.000000,456.000000,1262.000000,1346.000000
0,0.000000,0.000000,0.000000,603.870000,807.741000,0.000000,0.000000,0.000000,729.273000,780.252000,...,0.000000,0.000000,0.000000,0.000000,756.677000,0.000000,0.000000,0.000000,774.353000,0.000000
1,0.000000,0.000000,0.000000,149.390000,357.034000,0.000000,0.000000,0.000000,37.783500,400.121000,...,0.000000,0.000000,0.000000,0.000000,415.787000,0.000000,0.000000,0.000000,257.103000,0.000000
2,0.000000,0.000000,0.000000,0.776969,0.518555,0.000000,0.000000,0.000000,0.761039,0.375038,...,0.000000,0.000000,0.000000,0.000000,0.594527,0.000000,0.000000,0.000000,0.149461,0.000000
3,0.000000,0.000000,0.000000,549.068000,825.270000,0.000000,0.000000,0.000000,678.348000,594.124000,...,629.348000,0.000000,0.000000,617.618000,774.333000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,167.048000,364.919000,0.000000,0.000000,0.000000,74.942600,217.965000,...,5.365250,0.000000,0.000000,24.062700,529.467000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.334830,0.061986,0.000000,0.000000,0.000000,0.161580,0.063402,...,0.077537,0.000000,0.000000,0.279630,0.059947,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,366.833000,507.923000,263.057000,0.000000,0.000000,0.000000,445.229000,...,0.000000,0.000000,0.000000,427.590000,482.445000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,165.057000,251.259000,5.357470,0.000000,0.000000,0.000000,284.573000,...,0.000000,0.000000,0.000000,39.708200,372.733000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.193529,0.189982,0.070948,0.000000,0.000000,0.000000,0.112833,...,0.000000,0.000000,0.000000,0.125237,0.111511,0.000000,0.000000,0.000000,0.000000,0.000000


In [122]:
means_df = pd.DataFrame()

for name in labels_df['name']:
    if name != "warrior2_correct_1":
        mean = mean_ten_still_frames("train_data_csv/" + name + ".csv")
        means_df[name] = mean

In [123]:
means_df

,warrior2_knee1_1,warrior2_knee1_2,warrior2_knee1_3,warrior2_knee1_4,warrior2_knee1_5,warrior2_knee1_6,warrior2_knee2_1,warrior2_knee2_2,warrior2_knee2_3,warrior2_knee2_4,...,warrior2_torso_5,warrior2_hips_1,warrior2_hips_2,warrior2_hips_3,warrior2_hips_4,warrior2_hips_5,warrior2_correct_2,warrior2_correct_3,warrior2_correct_4,warrior2_correct_5
0,704.209400,713.547100,726.861800,668.605100,744.955100,752.005900,680.266900,643.079200,633.231200,668.573100,...,601.905800,790.036500,739.089900,836.994900,729.554100,742.727800,695.984400,681.864800,700.901400,670.142300
1,131.781800,175.237300,157.817600,165.122400,206.409600,178.844800,151.775900,163.040300,168.993500,210.749000,...,147.498400,132.718400,99.210630,137.657900,135.490700,141.152500,143.531600,145.662700,143.898800,147.347900
2,0.873184,0.896677,0.868085,0.914809,0.916538,0.909109,0.905208,0.884368,0.878988,0.924529,...,0.960221,0.912947,0.863128,0.889248,0.899125,0.857846,0.899138,0.945415,0.838789,0.898392
3,652.134200,662.675700,668.612900,674.037800,711.017000,699.518900,633.214900,633.258800,633.347300,640.943400,...,601.128000,729.321800,680.303200,774.343900,697.954200,696.417000,692.077400,648.720500,652.899100,677.934400
4,196.410600,239.264200,210.144300,229.722100,257.071600,254.365000,222.443000,214.102800,227.729600,259.184900,...,225.705600,182.528200,155.271000,196.407300,192.487400,200.134800,194.447100,212.170700,210.078800,206.407600
5,0.881475,0.817547,0.861277,0.842912,0.865699,0.855152,0.830091,0.886188,0.928193,0.840075,...,0.882708,0.860008,0.859764,0.895303,0.864776,0.844590,0.868649,0.897082,0.865459,0.839016
6,591.770400,598.018500,610.768900,615.650100,650.992000,655.644000,569.010500,570.642500,574.558600,586.212500,...,544.567800,675.981800,626.250100,722.626900,641.126400,668.594200,630.923100,599.995000,590.206300,617.652400
7,194.383900,229.744800,208.925600,228.597500,255.761200,241.471800,216.789700,213.105800,227.799400,259.159500,...,241.718000,168.421800,148.238500,187.794700,190.694900,182.720000,190.541700,210.741700,206.189300,208.160700
8,0.768179,0.738699,0.751413,0.805993,0.765953,0.793980,0.728016,0.823602,0.805227,0.774284,...,0.789988,0.729967,0.782260,0.742848,0.720570,0.788053,0.777970,0.785876,0.741953,0.822974
9,499.100400,495.393900,519.662600,531.819900,563.558800,556.879700,477.754300,480.716200,498.138000,490.470400,...,445.421300,585.460800,541.205700,636.849200,547.098700,583.331500,537.663800,494.445400,495.207100,539.023900


In [124]:
meansT = means_df.T.reset_index()
meansT = meansT.rename(columns={'index': 'name'})

In [125]:
meansT.head()

,name,0,1,2,3,4,5,6,7,8,...,65,66,67,68,69,70,71,72,73,74
0,warrior2_knee1_1,704.2094,131.7818,0.873184,652.1342,196.4106,0.881475,591.7704,194.3839,0.768179,...,0.822939,417.8212,608.2607,0.775085,409.9247,602.0120,0.791100,430.1526,586.2949,0.664928
1,warrior2_knee1_2,713.5471,175.2373,0.896677,662.6757,239.2642,0.817547,598.0185,229.7448,0.738699,...,0.808140,379.1896,646.9828,0.779803,368.7617,635.2954,0.804763,397.1880,615.4441,0.578325
2,warrior2_knee1_3,726.8618,157.8176,0.868085,668.6129,210.1443,0.861277,610.7689,208.9256,0.751413,...,0.802044,368.7948,637.2581,0.734101,363.1429,633.2472,0.678977,400.1497,605.1169,0.555250
3,warrior2_knee1_4,668.6051,165.1224,0.914809,674.0378,229.7221,0.842912,615.6501,228.5975,0.805993,...,0.766873,427.5490,645.0364,0.770176,413.9215,635.2317,0.800840,442.2568,614.6867,0.622250
4,warrior2_knee1_5,744.9551,206.4096,0.916538,711.0170,257.0716,0.865699,650.9920,255.7612,0.765953,...,0.816734,415.8755,641.7348,0.766126,407.2117,634.6370,0.770930,428.7204,619.5885,0.653987


In [126]:
poses_points = pd.merge(labels_df, meansT)

In [127]:
poses_points.shape

(63, 77)

In [129]:
poses_points

,name,label,0,1,2,3,4,5,6,7,...,65,66,67,68,69,70,71,72,73,74
0,warrior2_knee1_1,front_knee_acute,704.2094,131.78180,0.873184,652.1342,196.4106,0.881475,591.7704,194.3839,...,0.822939,417.8212,608.2607,0.775085,409.9247,602.0120,0.791100,430.1526,586.2949,0.664928
1,warrior2_knee1_2,front_knee_acute,713.5471,175.23730,0.896677,662.6757,239.2642,0.817547,598.0185,229.7448,...,0.808140,379.1896,646.9828,0.779803,368.7617,635.2954,0.804763,397.1880,615.4441,0.578325
2,warrior2_knee1_3,front_knee_acute,726.8618,157.81760,0.868085,668.6129,210.1443,0.861277,610.7689,208.9256,...,0.802044,368.7948,637.2581,0.734101,363.1429,633.2472,0.678977,400.1497,605.1169,0.555250
3,warrior2_knee1_3,torso_forward,726.8618,157.81760,0.868085,668.6129,210.1443,0.861277,610.7689,208.9256,...,0.802044,368.7948,637.2581,0.734101,363.1429,633.2472,0.678977,400.1497,605.1169,0.555250
4,warrior2_knee1_4,front_knee_acute,668.6051,165.12240,0.914809,674.0378,229.7221,0.842912,615.6501,228.5975,...,0.766873,427.5490,645.0364,0.770176,413.9215,635.2317,0.800840,442.2568,614.6867,0.622250
5,warrior2_knee1_4,head_front,668.6051,165.12240,0.914809,674.0378,229.7221,0.842912,615.6501,228.5975,...,0.766873,427.5490,645.0364,0.770176,413.9215,635.2317,0.800840,442.2568,614.6867,0.622250
6,warrior2_knee1_5,front_knee_acute,744.9551,206.40960,0.916538,711.0170,257.0716,0.865699,650.9920,255.7612,...,0.816734,415.8755,641.7348,0.766126,407.2117,634.6370,0.770930,428.7204,619.5885,0.653987
7,warrior2_knee1_5,front_knee_acute,744.9551,206.40960,0.916538,711.0170,257.0716,0.865699,650.9920,255.7612,...,0.816734,415.8755,641.7348,0.766126,407.2117,634.6370,0.770930,428.7204,619.5885,0.653987
8,warrior2_knee1_6,front_knee_acute,752.0059,178.84480,0.909109,699.5189,254.3650,0.855152,655.6440,241.4718,...,0.833031,425.8306,649.0227,0.832874,414.1195,639.9690,0.779706,439.5388,621.3464,0.644701
9,warrior2_knee1_6,hips_square,752.0059,178.84480,0.909109,699.5189,254.3650,0.855152,655.6440,241.4718,...,0.833031,425.8306,649.0227,0.832874,414.1195,639.9690,0.779706,439.5388,621.3464,0.644701


In [130]:
labels = labels_df['label'].unique()

In [131]:
labels 

array(['front_knee_acute', 'torso_forward', 'head_front', 'hips_square',
       'front_knee_obtuse', 'arms', 'shoulders_up', 'torso_backward',
       'step_wider', nan], dtype=object)

In [132]:
labels = list(labels[:-1])
labels

['front_knee_acute',
 'torso_forward',
 'head_front',
 'hips_square',
 'front_knee_obtuse',
 'arms',
 'shoulders_up',
 'torso_backward',
 'step_wider']

In [133]:
models = [(label, RandomForestClassifier()) for label in labels]

In [134]:
models[1][1]

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [145]:
dfs = []
correct_labels = []
predictions = []
accuracy_scores = []
probabilites = []
f_ones = []

for i in range(len(labels)):
    correct = poses_points[poses_points.label!=labels[i]]
    correct['label'] = 0
    incorrect = poses_points[poses_points.label==labels[i]]
    incorrect['label'] = 1
    df = pd.concat([correct, incorrect])
    dfs.append((labels[i], df))
    X = df.drop(['label', 'name'], axis=1)
    y = df['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    correct_labels.append((labels[i], y_test))
    models[i][1].fit(X_train, y_train)
    preds = models[i][1].predict(X_test)
    pred_probs = models[i][1].predict_proba(X_test)
    pred_probs = pred_probs.astype(np.float64)
    probabilites.append((labels[i], preds, pred_probs))
    y_test = y_test.astype(np.float64)
    predictions.append((labels[i], preds))
    score = accuracy_score(y_test, preds)
    accuracy_scores.append((labels[i], score))
    loss = f1_score(y_test, preds) 
    f_ones.append((labels[i], loss))
    print(labels[i], " model with accuarcy ", score, " and f1 score ", loss)
    
    

front_knee_acute  model with accuarcy  0.8095238095238095  and f1 score  0.0
torso_forward  model with accuarcy  0.9047619047619048  and f1 score  0.0
head_front  model with accuarcy  0.9047619047619048  and f1 score  0.0
hips_square  model with accuarcy  0.6666666666666666  and f1 score  0.0
front_knee_obtuse  model with accuarcy  0.8095238095238095  and f1 score  0.0
arms  model with accuarcy  0.6666666666666666  and f1 score  0.0
shoulders_up  model with accuarcy  0.7619047619047619  and f1 score  0.0
torso_backward  model with accuarcy  0.8571428571428571  and f1 score  0.0
step_wider  model with accuarcy  0.9047619047619048  and f1 score  0.0


In [146]:
correct_labels[0]

('front_knee_acute', 7     1
 1     1
 3     0
 50    0
 12    0
 43    0
 23    0
 19    0
 32    0
 6     1
 62    0
 16    0
 47    0
 20    0
 41    0
 56    0
 10    0
 24    0
 55    0
 15    0
 13    0
 Name: label, dtype: int64)

In [142]:
probabilites[0]

('front_knee_acute',
 array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([[0.575, 0.425],
        [0.525, 0.475],
        [0.2  , 0.8  ],
        [1.   , 0.   ],
        [0.975, 0.025],
        [1.   , 0.   ],
        [0.8  , 0.2  ],
        [1.   , 0.   ],
        [0.875, 0.125],
        [0.575, 0.425],
        [0.8  , 0.2  ],
        [0.925, 0.075],
        [1.   , 0.   ],
        [1.   , 0.   ],
        [1.   , 0.   ],
        [1.   , 0.   ],
        [1.   , 0.   ],
        [0.8  , 0.2  ],
        [1.   , 0.   ],
        [1.   , 0.   ],
        [0.975, 0.025]]))

In [144]:
dfs[0][1]

,name,label,0,1,2,3,4,5,6,7,...,65,66,67,68,69,70,71,72,73,74
3,warrior2_knee1_3,0,726.8618,157.81760,0.868085,668.6129,210.1443,0.861277,610.7689,208.9256,...,0.802044,368.7948,637.2581,0.734101,363.1429,633.2472,0.678977,400.1497,605.1169,0.555250
5,warrior2_knee1_4,0,668.6051,165.12240,0.914809,674.0378,229.7221,0.842912,615.6501,228.5975,...,0.766873,427.5490,645.0364,0.770176,413.9215,635.2317,0.800840,442.2568,614.6867,0.622250
9,warrior2_knee1_6,0,752.0059,178.84480,0.909109,699.5189,254.3650,0.855152,655.6440,241.4718,...,0.833031,425.8306,649.0227,0.832874,414.1195,639.9690,0.779706,439.5388,621.3464,0.644701
10,warrior2_knee2_1,0,680.2669,151.77590,0.905208,633.2149,222.4430,0.830091,569.0105,216.7897,...,0.795125,379.9539,642.3068,0.761749,367.0926,634.4633,0.800861,392.8928,611.3529,0.598381
11,warrior2_knee2_1,0,680.2669,151.77590,0.905208,633.2149,222.4430,0.830091,569.0105,216.7897,...,0.795125,379.9539,642.3068,0.761749,367.0926,634.4633,0.800861,392.8928,611.3529,0.598381
12,warrior2_knee2_2,0,643.0792,163.04030,0.884368,633.2588,214.1028,0.886188,570.6425,213.1058,...,0.836380,327.6777,637.2955,0.649516,321.8019,633.2425,0.659223,364.8675,611.9394,0.543324
13,warrior2_knee2_2,0,643.0792,163.04030,0.884368,633.2588,214.1028,0.886188,570.6425,213.1058,...,0.836380,327.6777,637.2955,0.649516,321.8019,633.2425,0.659223,364.8675,611.9394,0.543324
14,warrior2_knee2_3,0,633.2312,168.99350,0.878988,633.3473,227.7296,0.928193,574.5586,227.7994,...,0.825368,409.9702,635.2981,0.829454,398.2024,631.3044,0.780296,419.7545,607.8256,0.614476
15,warrior2_knee2_3,0,633.2312,168.99350,0.878988,633.3473,227.7296,0.928193,574.5586,227.7994,...,0.825368,409.9702,635.2981,0.829454,398.2024,631.3044,0.780296,419.7545,607.8256,0.614476
16,warrior2_knee2_4,0,668.5731,210.74900,0.924529,640.9434,259.1849,0.840075,586.2125,259.1595,...,0.837206,368.2080,652.9032,0.822715,355.1116,648.9588,0.808750,365.8417,621.7635,0.648724


In [153]:
# equally balanced data
models = [(label, RandomForestClassifier()) for label in labels]
dfs = []
correct_labels = []
predictions = []
accuracy_scores = []
probabilites = []
f_ones = []

for i in range(len(labels)):
    correct = poses_points[poses_points.label!=labels[i]]
    correct['label'] = 0
    incorrect = poses_points[poses_points.label==labels[i]].iloc[0:len(correct), :]
    incorrect['label'] = 1
    df = pd.concat([correct, incorrect])
    dfs.append((labels[i], df))
    X = df.drop(['label', 'name'], axis=1)
    y = df['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    correct_labels.append((labels[i], y_test))
    models[i][1].fit(X_train, y_train)
    preds = models[i][1].predict(X_test)
    pred_probs = models[i][1].predict_proba(X_test)
    pred_probs = pred_probs.astype(np.float64)
    probabilites.append((labels[i], preds, pred_probs))
    y_test = y_test.astype(np.float64)
    predictions.append((labels[i], preds))
    score = accuracy_score(y_test, preds)
    accuracy_scores.append((labels[i], score))
    loss = f1_score(y_test, preds) 
    f_ones.append((labels[i], loss))
    print(labels[i], " model with accuarcy ", score, " and f1 score ", loss)

front_knee_acute  model with accuarcy  0.7142857142857143  and f1 score  0.0
torso_forward  model with accuarcy  0.9047619047619048  and f1 score  0.0
head_front  model with accuarcy  0.9047619047619048  and f1 score  0.0
hips_square  model with accuarcy  0.6190476190476191  and f1 score  0.0
front_knee_obtuse  model with accuarcy  0.8095238095238095  and f1 score  0.0
arms  model with accuarcy  0.7142857142857143  and f1 score  0.0
shoulders_up  model with accuarcy  0.8095238095238095  and f1 score  0.0
torso_backward  model with accuarcy  0.8571428571428571  and f1 score  0.0
step_wider  model with accuarcy  0.9047619047619048  and f1 score  0.0
